<a href="https://colab.research.google.com/github/ellisedp/ellisedp.github.io/blob/master/Data_Mining_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np  # For euclidiean distance: np.linalg.norm(point1 - point2) 
import math
from scipy.stats import zscore

# Import and Preprocess Data

Import Testing Data

In [ ]:
# Import testing data 
dftest = pd.read_csv('spam_test.csv')
dftest

,ID,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f49,f50,f51,f52,f53,f54,f55,f56,f57,Label
0,t1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.50,0.00,...,0.000,0.182,0.0,0.000,0.182,0.000,3.545,21,78,1
1,t2,0.31,0.00,0.63,1.91,0.21,0.00,0.00,0.00,0.42,...,0.000,0.169,0.0,0.358,0.188,0.075,2.847,75,447,1
2,t3,0.00,0.75,0.37,0.00,0.00,0.00,0.75,0.00,0.00,...,0.000,0.000,0.0,0.242,0.000,0.145,9.584,332,508,1
3,t4,0.00,1.96,0.98,0.00,0.00,0.00,1.96,0.00,0.00,...,0.000,0.168,0.0,1.011,0.000,0.000,2.888,12,52,1
4,t5,0.51,0.43,0.29,0.00,0.14,0.03,0.00,0.18,0.54,...,0.012,0.080,0.0,0.454,0.523,0.136,6.590,739,2333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,t2297,0.31,0.00,0.62,0.00,0.00,0.31,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
2297,t2298,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
2298,t2299,0.30,0.00,0.30,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
2299,t2300,0.96,0.00,0.00,0.00,0.32,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


Preprocess Testing Data

In [ ]:
# Check for NaN
dftest.isnull().sum().sum()

0

In [ ]:
# Count labels 
dftest.Label.value_counts()

0    1394
1     907
Name: Label, dtype: int64

In [ ]:
# Turn pandas df into a numpy array
dftest_array = dftest.to_numpy()
print(dftest_array)
print(dftest_array.shape)

[['t1' 0.0 0.0 ... 21 78 1]
 ['t2' 0.31 0.0 ... 75 447 1]
 ['t3' 0.0 0.75 ... 332 508 1]
 ...
 ['t2299' 0.3 0.0 ... 6 118 0]
 ['t2300' 0.96 0.0 ... 5 78 0]
 ['t2301' 0.0 0.0 ... 5 40 0]]
(2301, 59)


In [ ]:
# Make array with only features 
dftest_feat = np.delete(dftest_array, [0, 58], 1)
print(dftest_feat)
print(dftest_feat.shape)

[[0.0 0.0 0.0 ... 3.545 21 78]
 [0.31 0.0 0.63 ... 2.847 75 447]
 [0.0 0.75 0.37 ... 9.584 332 508]
 ...
 [0.3 0.0 0.3 ... 1.404 6 118]
 [0.96 0.0 0.0 ... 1.147 5 78]
 [0.0 0.0 0.65 ... 1.25 5 40]]
(2301, 57)


In [ ]:
# Normalize testing data
DFtest = dftest.drop(columns=[" ID", "Label"])
N_dftest = DFtest.apply(zscore)
N_dftest_feat = N_dftest.to_numpy()

array([[-0.33949281, -0.27355124, -0.55156119, ..., -0.04382859,
        -0.13165149, -0.31213475],
       [ 0.5981446 , -0.27355124,  0.69037676, ..., -0.084122  ,
         0.09770585,  0.28698167],
       [-0.33949281,  1.69673696,  0.17783094, ...,  0.30478452,
         1.18927693,  0.38602259],
       ...,
       [ 0.56789823, -0.27355124,  0.03983783, ..., -0.16742201,
        -0.19536187, -0.24718988],
       [ 2.56415852, -0.27355124, -0.55156119, ..., -0.18225783,
        -0.19960923, -0.31213475],
       [-0.33949281, -0.27355124,  0.72980336, ..., -0.17631196,
        -0.19960923, -0.37383238]])

Import Training Data

In [ ]:
dftrain = pd.read_csv('spam_train.csv')
dftrain

,ID,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f49,f50,f51,f52,f53,f54,f55,f56,f57,Label
0,x1,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,x2,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,x3,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,x4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,x5,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,x2296,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.250,0.0,0.000,0.000,0.000,1.601,8,173,0
2296,x2297,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.632,0.0,0.000,0.000,0.000,2.142,11,45,0
2297,x2298,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.091,0.0,0.045,0.000,0.000,2.333,15,175,0
2298,x2299,0.00,0.00,0.60,0.0,0.60,0.00,0.00,0.00,0.00,...,0.00,0.110,0.0,0.000,0.000,0.000,1.074,3,29,0


Preprocess Training Data

In [ ]:
# Check for NaN
dftrain.isnull().sum().sum()

0

In [ ]:
# Count labels 
dftrain.Label.value_counts()

0    1394
1     906
Name: Label, dtype: int64

In [ ]:
# Turn pandas df into a numpy array
dftrain_array = dftrain.to_numpy()
print(dftrain_array)
print(dftrain_array.shape)

[['x1' 0.0 0.64 ... 61 278 1]
 ['x2' 0.21 0.28 ... 101 1028 1]
 ['x3' 0.06 0.0 ... 485 2259 1]
 ...
 ['x2298' 0.0 0.0 ... 15 175 0]
 ['x2299' 0.0 0.0 ... 3 29 0]
 ['x2300' 0.0 0.32 ... 157 772 0]]
(2300, 59)


In [ ]:
# Make array with only features 
dftrain_feat = np.delete(dftrain_array, [0, 58], 1)
print(dftrain_feat)
print(dftrain_feat.shape)

[[0.0 0.64 0.64 ... 3.756 61 278]
 [0.21 0.28 0.5 ... 5.114 101 1028]
 [0.06 0.0 0.71 ... 9.821 485 2259]
 ...
 [0.0 0.0 0.0 ... 2.333 15 175]
 [0.0 0.0 0.6 ... 1.074 3 29]
 [0.0 0.32 0.32 ... 3.918 157 772]]
(2300, 57)


In [ ]:
# Normalize training data
DFtrain = dftrain.drop(columns=[" ID", "Label"])
N_dftrain = DFtrain.apply(zscore)
N_dftrain_feat = N_dftrain.to_numpy()

array([[-3.49034620e-01,  1.78848168e-01,  7.15694977e-01, ...,
        -5.61520521e-02,  6.04041908e-02, -3.07579604e-02],
       [ 4.07692741e-01, -2.35882426e-02,  4.36187660e-01, ...,
        -2.33287294e-02,  3.39715889e-01,  1.22718414e+00],
       [-1.32826803e-01, -1.81038784e-01,  8.55448636e-01, ...,
         9.04410643e-02,  3.02110819e+00,  3.29188643e+00],
       ...,
       [-3.49034620e-01, -1.81038784e-01, -5.62052756e-01, ...,
        -9.05464470e-02, -2.60804262e-01, -2.03515342e-01],
       [-3.49034620e-01, -1.81038784e-01,  6.35835744e-01, ...,
        -1.20976906e-01, -3.44597772e-01, -4.48394736e-01],
       [-3.49034620e-01, -1.09530812e-03,  7.68211103e-02, ...,
        -5.22364569e-02,  7.30752267e-01,  7.97806567e-01]])

# Non-normalized

## Calculate distance between each email in the test set and each email in the training set
Then store in a 2D array (dist_array)

In [ ]:
count = 0

# Create array to store all distances 
dist_array = np.zeros(((len(dftest_feat)), (len(dftrain_feat))))

for i in range(len(dftest_feat)): 
  A = dftest_feat[i,:]
  for j in range(len(dftrain_feat)): 
    # Get each row of the dfs as arrays 
    B = dftrain_feat[j,:]

    # Calculate distance between two emails 
    dist = np.linalg.norm(A-B)
    #print(dist)

    # Store distances in new array
    dist_array[i,j] = dist

    count += 1

print(count)

5292300


## Find k shortest distances

In [ ]:
# Make all classes from dftest a separate array
test_Labels = dftest[["Label"]].to_numpy()
test_Labels.shape = (1, 2301)

# Create 2D array to store all class predictions for the test data
# Rows are equal to k = [0,1,2,3,4,5,6,7,8,9]
predicted_class = np.empty([10, 2301])

row_for_k = [0,1,2,3,4,5,6,7,8,9]
k_array= [1, 5, 11, 21, 41, 61, 81, 101, 201, 401]

for k in row_for_k:
  curr_k = k_array[k]
  for i in range(0,2301):
    classes = [] # Classes of neighbor emails
    # Make temp dictionary with distances sorted and corresponding training email#
    row_dict = {}
    for j in range(0,2300):
      row_dict[j] = dist_array[i,j]
    sorted_dict = dict(sorted(row_dict.items(), key=lambda item: item[1]))
    # Now I have the distances from shortest to largest with their training emails

    for c in range(curr_k):
      key_list = list(sorted_dict.keys())
      train_email = key_list[c]

      # Row containing training email with shortest dist to test email
      closest = dftrain_array[(train_email-1),:]

      # Array containing the classes of the closest emails
      classes.append(closest[-1])

    #print(classes)

    # Using majority vote of of k nearest neighbors, determine class of email
    if np.size(classes) == 1:
      if np.sum(classes) == 0:
        #print("0: Not Spam") 
        predicted_class[k,i] = 0
      elif np.sum(classes) == 1:
        #print("1: Spam")
        predicted_class[k,i] = 1
    elif np.sum(classes) < ((np.size(classes))/2):
      #print("0: Not Spam")
      predicted_class[k,i] = 0
    elif np.sum(classes) > ((np.size(classes))/2):
      #print("1: Spam")
      predicted_class[k,i] = 1


In [ ]:
# Test accuracy 
k_array= [1, 5, 11, 21, 41, 61, 81, 101, 201, 401]

for k in range(0,10):
  curr_k = k_array[k]
  correct = 0
  for t in range(2301):        
    if predicted_class[k,t] == test_Labels[:,t]:
      correct += 1
  accuracy = correct/2300
  print("Accuracy of predicted class results when k =", curr_k, ":", accuracy)

Accuracy of predicted class results when k = 1 : 0.7526086956521739
Accuracy of predicted class results when k = 5 : 0.7552173913043478
Accuracy of predicted class results when k = 11 : 0.7652173913043478
Accuracy of predicted class results when k = 21 : 0.7469565217391304
Accuracy of predicted class results when k = 41 : 0.7526086956521739
Accuracy of predicted class results when k = 61 : 0.7378260869565217
Accuracy of predicted class results when k = 81 : 0.7269565217391304
Accuracy of predicted class results when k = 101 : 0.7291304347826087
Accuracy of predicted class results when k = 201 : 0.73
Accuracy of predicted class results when k = 401 : 0.7204347826086956


# Normalized 

## Calculate distance between each email in the test set and each email in the training set (norm)
Then store in a 2D array (dist_array)

In [ ]:
count = 0

# Create array to store all distances 
N_dist_array = np.zeros(((len(N_dftest_feat)), (len(N_dftrain_feat))))

for i in range(len(N_dftest_feat)): 
  A = N_dftest_feat[i,:]
  for j in range(len(N_dftrain_feat)): 
    # Get each row of the dfs as arrays 
    B = N_dftrain_feat[j,:]

    # Calculate distance between two emails 
    N_dist = np.linalg.norm(A-B)
    
    # Store distances in new array
    N_dist_array[i,j] = N_dist

    count += 1

print(count)

5292300


## Find k shortest distances (norm)

In [ ]:
# Make all classes from dftest a separate array
N_test_Labels = dftest[["Label"]].to_numpy()
N_test_Labels.shape = (1, 2301)

# Create 2D array to store all class predictions for the test data
# Rows are equal to k = [0,1,2,3,4,5,6,7,8,9]
N_predicted_class = np.empty([10, 2301])

row_for_k = [0,1,2,3,4,5,6,7,8,9]
k_array= [1, 5, 11, 21, 41, 61, 81, 101, 201, 401]

for k in row_for_k:
  curr_k = k_array[k]
  for i in range(0,2301):
    N_classes = [] # Classes of neighbor emails
    # Make temp dictionary with distances sorted and corresponding training email#
    row_dict = {}
    for j in range(0,2300):
      row_dict[j] = N_dist_array[i,j]
    sorted_dict = dict(sorted(row_dict.items(), key=lambda item: item[1]))
    # Now I have the distances from shortest to largest with their training emails

    for c in range(curr_k):
      key_list = list(sorted_dict.keys())
      train_email = key_list[c]

      # Row containing training email with shortest dist to test email
      N_closest = dftrain_array[(train_email-1),:]

      # Array containing the classes of the closest emails
      N_classes.append(N_closest[-1])

    #print(N_classes)

    # Using majority vote of of k nearest neighbors, determine class of email
    if np.size(N_classes) == 1:
      if np.sum(N_classes) == 0:
        #print("0: Not Spam") 
        N_predicted_class[k,i] = 0
      elif np.sum(N_classes) == 1:
        #print("1: Spam")
        N_predicted_class[k,i] = 1
    elif np.sum(N_classes) < ((np.size(N_classes))/2):
      #print("0: Not Spam")
      N_predicted_class[k,i] = 0
    elif np.sum(N_classes) > ((np.size(N_classes))/2):
      #print("1: Spam")
      N_predicted_class[k,i] = 1


In [ ]:
# Test accuracy 
k_array= [1, 5, 11, 21, 41, 61, 81, 101, 201, 401]

for k in range(0,10):
  curr_k = k_array[k]
  correct = 0
  for t in range(2301):        
    if N_predicted_class[k,t] == N_test_Labels[:,t]:
      correct += 1
  accuracy = correct/2300
  print("Accuracy of predicted class results when k =", curr_k, ":", accuracy, "(normalized)")

Accuracy of predicted class results when k = 1 : 0.8234782608695652 (normalized)
Accuracy of predicted class results when k = 5 : 0.8326086956521739 (normalized)
Accuracy of predicted class results when k = 11 : 0.8752173913043478 (normalized)
Accuracy of predicted class results when k = 21 : 0.871304347826087 (normalized)
Accuracy of predicted class results when k = 41 : 0.871304347826087 (normalized)
Accuracy of predicted class results when k = 61 : 0.8726086956521739 (normalized)
Accuracy of predicted class results when k = 81 : 0.8708695652173913 (normalized)
Accuracy of predicted class results when k = 101 : 0.8647826086956522 (normalized)
Accuracy of predicted class results when k = 201 : 0.8469565217391304 (normalized)
Accuracy of predicted class results when k = 401 : 0.8139130434782609 (normalized)


## KNN predicted labels for the first 50 instances 

In [ ]:
for i in range(0,51):
  print("t", (i+1), "[", end = ' ')
  for j in range(0,10):
    if N_predicted_class[j,i] == 1.0:
      print("spam", end = ' ')
    else:
      print("no", end = ' ')
  print("]")

t 1 [ spam spam spam spam spam no no no no no ]
t 2 [ spam spam spam spam spam spam spam no no no ]
t 3 [ spam spam spam spam spam spam spam spam spam spam ]
t 4 [ spam spam spam spam no no spam spam spam spam ]
t 5 [ spam spam spam spam spam spam spam spam spam spam ]
t 6 [ spam spam spam no no spam spam spam spam spam ]
t 7 [ spam no no no no no no no no no ]
t 8 [ spam spam spam spam spam spam spam spam spam spam ]
t 9 [ spam spam spam spam spam spam spam spam spam spam ]
t 10 [ spam spam spam spam spam spam spam spam spam spam ]
t 11 [ spam spam spam spam spam spam spam spam spam spam ]
t 12 [ spam spam spam spam spam spam spam spam spam spam ]
t 13 [ spam spam spam spam spam spam no no no no ]
t 14 [ no spam spam spam no no no no no no ]
t 15 [ spam spam spam spam spam spam spam spam spam spam ]
t 16 [ spam spam spam spam spam spam spam spam spam spam ]
t 17 [ spam spam spam spam spam spam spam spam spam spam ]
t 18 [ spam spam spam spam spam spam spam no no no ]
t 19 [ spam spam 